In [1]:
import pandas
import json
import re
from os.path import join, isdir
from os import mkdir

In [2]:
trump_df_name = "../DT_data.csv"
hillary_df_name = "../HC_data.csv"

trump_df = pandas.read_csv(trump_df_name)
hillary_df = pandas.read_csv(hillary_df_name)

In [3]:
SEQ_LENGTH = 512

def write_tweets(candidate, df, column, folder, file_type):

  if not isdir(folder):
    mkdir(folder)

  for i in range(df[column].shape[0]):
    tweets = json.loads(re.sub('\'', '\"',  df[column][i]))
    day = df["DATE"][i]
    category = column.lower()
    index = 1
    for tweet in tweets:
      article_name = candidate + "_" + category + "_" + str(i) + "_" + str(index) + "." + file_type
      index += 1
      with open(join(folder, article_name), "w") as f:

#### Write in aschern processing format
        if file_type == "txt":
          f.write(tweet["text"])
          f.write("\n\n")
#### Write JSON
        elif file_type == "json":
          tweet["date"] = day
          json.dump(tweet, f)

def write_non_tweets(candidate, df, non_tweet_folder, file_type, aschern=False):
  for i in range(df["NON_TWEETS"].shape[0]):
    if df["NON_TWEETS"][i] == []:
      continue
    category = df["CONTENT_CATEGORY"][i]
    if type(category) == float or not category or category == "tweet":
      continue
    folder = non_tweet_folder
    if not aschern:
      folder = join(non_tweet_folder, category)
    if not isdir(folder):
      mkdir(folder)
    article_name = candidate + "_" + category + "_" + str(i) + "." + file_type
    with open(folder + "/" + article_name, "w") as f:

#### Write in aschern processing format
      if file_type == "txt":
        #TODO: SPLIT BY SENTENCES "/s, ./s ...
        
        text = df["NON_TWEETS"][i]
        sentences = re.split('[\'\".]\s', text)

        # if len(text) <= SEQ_LENGTH:
        #   return [text]
        
        # sentence_start = 0
        # sentence_end = SEQ_LENGTH - 1
        # sentences = []

        # eos = re.compile(r"([\'\"\.\?\!])(\s|\t)")
        # space = re.compile(r"\s")
        
        # while sentence_end < len(text):
        #   if (eos.match(text[sentence_end-1:sentence_end+1]) or (sentence_end-sentence_start < SEQ_LENGTH - 100 and space.match(text[sentence_end:sentence_end+1]))):
        #     sentence = text[sentence_start:sentence_end]
        #     sentences.append(sentence)
        #     sentence_start = sentence_end + 1
        #     sentence_end += SEQ_LENGTH - 1
        #   else:
        #     sentence_end -= 1

        for sentence in sentences:
          if len(sentence.split()) > 4:
            f.write(sentence + "\n")
        f.write("\n\n")

#### Write JSON
      elif file_type == "json":
        data = {"text": df["NON_TWEETS"][i], "date": df["DATE"][i]}
        json.dump(data, f)
      
    # print("Processed file: " + str(i))

      

In [5]:
from pathlib import Path

base_folder = "../2016_election_data"

#TODO: Delete Folder
def rmdir(directory):
  directory = Path(directory)
  for item in directory.iterdir():
      if item.is_dir():
        rmdir(item)
      else:
        item.unlink()
  directory.rmdir()

dt_directory = "donald_trump_campaign"
hc_directory = "hillary_clinton_campaign"

rmdir(join("../2016_election_data", dt_directory))
rmdir(join("../2016_election_data", hc_directory))
rmdir("../aschern_data")

Path(join(base_folder, dt_directory, "donald_trump")).mkdir(parents=True, exist_ok=True)
Path(join(base_folder, hc_directory, "hillary_clinton")).mkdir(parents=True, exist_ok=True)

# Create folder structure with JSON documents

In [6]:
write_tweets("HC", hillary_df, "TWEETS", join(base_folder, "hillary_clinton_campaign/hillary_clinton/tweets"), "json")
write_tweets("HC", hillary_df, "RETWEETS", join(base_folder, "hillary_clinton_campaign/hillary_clinton/retweets"), "json")
write_tweets("DT", trump_df, "TWEETS", join(base_folder, "donald_trump_campaign/donald_trump/tweets"), "json")
write_tweets("DT", trump_df, "RETWEETS", join(base_folder, "donald_trump_campaign/donald_trump/retweets"), "json")

In [7]:
write_non_tweets("HC", hillary_df, join(base_folder, "hillary_clinton_campaign/hillary_clinton"), "json")
write_non_tweets("DT", trump_df, join(base_folder, "donald_trump_campaign/donald_trump"), "json")

# Create folder structure for aschern model

In [8]:
write_tweets("HC", hillary_df, "TWEETS", "../aschern_data", "txt")
write_tweets("HC", hillary_df, "RETWEETS", "../aschern_data", "txt")
write_tweets("DT", trump_df, "TWEETS", "../aschern_data", "txt")
write_tweets("DT", trump_df, "RETWEETS", "../aschern_data", "txt")

In [9]:
write_non_tweets("HC", hillary_df, "../aschern_data", "txt", aschern=True)
write_non_tweets("DT", trump_df, "../aschern_data", "txt", aschern=True)